In [9]:
from utils.helpers import *
from utils.preprocess_data import *

In [10]:
import pandas as pd

data = pd.read_csv("./augment_data.csv", encoding="utf8")
data.incorrect = data.incorrect.str.strip()
data.correct = data.correct.str.strip()

In [11]:
data.head(5)

,incorrect,correct
0,"Hkacs với những lần trươ1c, tàu họ đã được tàu...","Khác với những lần trước, tàu họ đã được tàu t..."
1,Cầu thủ mang về bàn ghỡ hòa cho đội khách Tcho...,Cầu thủ mang về bàn gỡ hòa cho đội khách Tchoy...
2,Sau đó tôi dô4 thế nào nó cũng chỉ nghi một ch...,Sau đó tôi dỗ thế nào nó cũng chỉ ngồi một chỗ...
3,ĐB của Đồng bằng sụng Cửu Long này xúc động gl...,ĐB của Đồng bằng sông Cửu Long này xúc động nó...
4,Mạch điều jgifren trung tâm.,Mạch điều khiển trung tâm.


In [12]:
def process_labels(correct, incorrect):
    _, labels = perfect_align(incorrect, correct, 2)
        
    final_label = []
    skip_flag = False
    for idx in range(len(labels)):
        label = labels[idx]
        if idx + 1 < len(labels):
            next_label = labels[idx+1]
            if label[0].startswith("KEEP") and next_label[0].startswith("APPEND"):
                final_label.append([next_label[0], label[1]])
                skip_flag = True
                if idx + 1 == len(labels) - 1:
                    break
            elif label[0].startswith("APPEND") and next_label[0].startswith("APPEND"):
                tmp_label, tmp_e = final_label[-1]
                tmp_label = tmp_label + next_label[0][-1]
                final_label[-1] = [tmp_label, tmp_e]
                if idx + 1 == len(labels) - 1:
                    break
            elif skip_flag:
                skip_flag = False
            else:
                final_label.append(label)
                skip_flag = False
        else:
            final_label.append(label)
    
    assert len(final_label)==len(incorrect), f"Length labels not equal for sentence: {incorrect}"
    return [label[0] for label in final_label]

In [6]:
from tqdm import tqdm

def make_char_label(data):
    corrects = data.correct.values.tolist()
    incorrects = data.incorrect.values.tolist()
    labels = []
    bar = tqdm(total=len(corrects))
    for correct, incorrect in zip(corrects, incorrects):
        labels.append(process_labels(correct, incorrect))
        bar.update(1)
    data["labels"] = labels
    return data

In [15]:
corrects = data.correct.values.tolist()
incorrects = data.incorrect.values.tolist()

In [13]:
process_labels("hôm nay tôi tới trường, bị bắt nạt nên buồn quá!", "hoom ni tui tới chường bị bắt lạt nên bn wá!")

['KEEP',
 'DELETE',
 'REPLACE_ô',
 'KEEP',
 'KEEP',
 'APPEND_a',
 'REPLACE_y',
 'KEEP',
 'KEEP',
 'REPLACE_ô',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'REPLACE_t',
 'REPLACE_r',
 'KEEP',
 'KEEP',
 'KEEP',
 'APPEND_,',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'REPLACE_n',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'APPEND_uồ',
 'KEEP',
 'APPEND_q',
 'REPLACE_u',
 'KEEP',
 'KEEP']

In [16]:
process_labels(corrects[0], incorrects[0])

['REPLACE_K',
 'REPLACE_h',
 'REPLACE_á',
 'KEEP',
 'DELETE',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'DELETE',
 'REPLACE_ớ',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'DELETE',
 'REPLACE_ầ',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'REPLACE_T',
 'REPLACE_r',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'DELETE',
 'REPLACE_ò',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP'

In [17]:
import pickle 

df = pickle.load(open("labeled_dataset.pkl", "rb"))